In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#load dataset
df=pd.read_csv('train.csv')

In [5]:
#show data
df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
x=df.drop(["MiscFeature"], axis=1)

In [7]:
x.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,0,9,2008,WD,Normal,223500


In [30]:
#dimentionality reduction can be helpful for this dataset
#figure out the feature
a=df.columns.tolist()
drp_lst=['MSSubClass','MSZoning','LotFrontage','Alley','LandContour','LotConfig','Condition1', 'Condition2',
         'BldgType','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','ExterQual','ExterCond',
        'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF',
        'Heating','HeatingQC','CentralAir','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath',
        'BsmtHalfBath','HalfBath','KitchenQual','Fireplaces','FireplaceQu','GarageType','GarageYrBlt','GarageFinish',
        'GarageCars','GarageArea','GarageQual','GarageCond','PavedDrive','WoodDeckSF','OpenPorchSF','EnclosedPorch',
        '3SsnPorch','ScreenPorch','PoolArea','PoolQC','Fence','MiscFeature','LotShape','LandSlope','Functional',
        'MoSold','YrSold','SaleType','SaleCondition','Utilities','HouseStyle','TotRmsAbvGrd','Id']


In [31]:
#drop
x= df.drop(drp_lst, axis=1)
x.head(5)

,LotArea,Street,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Foundation,TotalBsmtSF,Electrical,FullBath,BedroomAbvGr,KitchenAbvGr,MiscVal,SalePrice
0,8450,Pave,CollgCr,7,5,2003,2003,PConc,856,SBrkr,2,3,1,0,208500
1,9600,Pave,Veenker,6,8,1976,1976,CBlock,1262,SBrkr,2,3,1,0,181500
2,11250,Pave,CollgCr,7,5,2001,2002,PConc,920,SBrkr,2,3,1,0,223500
3,9550,Pave,Crawfor,7,5,1915,1970,BrkTil,756,SBrkr,1,3,1,0,140000
4,14260,Pave,NoRidge,8,5,2000,2000,PConc,1145,SBrkr,2,4,1,0,250000


In [32]:
x.rename(columns={'Neighborhood':'NeighborhoodArea','FullBath':'Bath','BedroomAbvGr':'BedRm',
                 'KitchenAbvGr':'kitchen'}, inplace=True)
x.head(3)

,LotArea,Street,NeighborhoodArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,Foundation,TotalBsmtSF,Electrical,Bath,BedRm,kitchen,MiscVal,SalePrice
0,8450,Pave,CollgCr,7,5,2003,2003,PConc,856,SBrkr,2,3,1,0,208500
1,9600,Pave,Veenker,6,8,1976,1976,CBlock,1262,SBrkr,2,3,1,0,181500
2,11250,Pave,CollgCr,7,5,2001,2002,PConc,920,SBrkr,2,3,1,0,223500
